In [ ]:
!pip install mysql-connector-python

     |████████████████████████████████| 37.6 MB 1.4 MB/s 


In [ ]:
import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'root',
    'password': 'itsnotApassword',
    'host': '44.44.44.444',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'ssl/server-ca.pem',
    'ssl_cert': 'ssl/client-cert.pem',
    'ssl_key': 'ssl/client-key.pem'
}

# now we establish our connection
cnxn = mysql.connector.connect(**config)

In [ ]:
config['database'] = 'ytdatabase'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Space_Corrected.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Company Name', 'Location', 'Datum',
       'Detail', 'Status Rocket', ' Rocket', 'Status Mission'],
      dtype='object')

In [ ]:
#The heck are those two unnamed columns? Gotcha drop em all!
col_names_list = ['Company Name', 'Location', 'Datum', 'Detail', 'Status Rocket', ' Rocket', 'Status Mission']
df = pd.read_csv('Space_Corrected.csv', usecols = col_names_list)
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [ ]:
df.dtypes

Company Name      object
Location          object
Datum             object
Detail            object
Status Rocket     object
Rocket            object
Status Mission    object
dtype: object

In [ ]:
#For the sake of JUST showing how to import CSV file as a Table in remote Cloud Database I will skip converting these datatypes to it's proper ones
#I'm a lazy programmer*

In [ ]:
cursor.execute("CREATE TABLE space_missions ("
               "company_name VARCHAR(255),"
               "location VARCHAR(255),"
               "datum VARCHAR(255),"
               "detail VARCHAR(255),"
               "status_rocket VARCHAR(255),"
               "rocket VARCHAR(255),"
               "status_mission VARCHAR(255) )")

cnxn.commit()

In [ ]:
#First we setup our query
query = ("INSERT INTO space_missions (company_name, location, datum, detail, status_rocket, rocket, status_mission) "
         "VALUES (%s, %s, %s, %s, %s, %s, %s)")

#and then we execute with every row in our dataframe
cursor.executemany(query, list(df.to_records(index=False)))
cnxn.commit()  # and commit changes

InterfaceError: ignored

In [ ]:
#This no works because.... 
type(list(df.to_records(index=False))[0])

numpy.record

In [ ]:
#StackOverflow to the rescue.... 
list(df.itertuples(index=False, name = None))[0]

('SpaceX',
 'LC-39A, Kennedy Space Center, Florida, USA',
 'Fri Aug 07, 2020 05:12 UTC',
 'Falcon 9 Block 5 | Starlink V1 L9 & BlackSky',
 'StatusActive',
 '50.0 ',
 'Success')

In [ ]:
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [ ]:
#Here I had some problems with NaNs in Rocket column. First my Rocket column had type FLOAT() but... later I found out that
#I might check data types of those columns :P 
#Oh, yeah! I converted them to zeroes but had other problems since it is not type of int

In [ ]:
df.isnull().sum()

Company Name         0
Location             0
Datum                0
Detail               0
Status Rocket        0
 Rocket           3360
Status Mission       0
dtype: int64

In [ ]:
df[' Rocket'] = df[' Rocket'].fillna(0)

In [ ]:
df.isnull().sum()

Company Name      0
Location          0
Datum             0
Detail            0
Status Rocket     0
 Rocket           0
Status Mission    0
dtype: int64

In [ ]:
df = df.rename(columns={' Rocket': 'Rocket'})

In [ ]:
df.columns

Index(['Company Name', 'Location', 'Datum', 'Detail', 'Status Rocket',
       'Rocket', 'Status Mission'],
      dtype='object')

In [ ]:
list(df.itertuples(index=False, name = None))[1]

('CASC',
 'Site 9401 (SLS-2), Jiuquan Satellite Launch Center, China',
 'Thu Aug 06, 2020 04:01 UTC',
 'Long March 2D | Gaofen-9 04 & Q-SAT',
 'StatusActive',
 '29.75 ',
 'Success')

In [ ]:
query = ("INSERT INTO space_missions (company_name, location, datum, detail, status_rocket, rocket, status_mission) "
         "VALUES (%s, %s, %s, %s, %s, %s, %s)")

cursor.executemany(query, list(df.itertuples(index=False, name = None)))
cnxn.commit()  # and commit changes

In [ ]:
cursor.execute('SELECT * FROM space_missions LIMIT 10')
output = cursor.fetchall()

In [ ]:
for row in output:
  print(row)

('SpaceX', 'LC-39A, Kennedy Space Center, Florida, USA', 'Fri Aug 07, 2020 05:12 UTC', 'Falcon 9 Block 5 | Starlink V1 L9 & BlackSky', 'StatusActive', '50.0 ', 'Success')
('CASC', 'Site 9401 (SLS-2), Jiuquan Satellite Launch Center, China', 'Thu Aug 06, 2020 04:01 UTC', 'Long March 2D | Gaofen-9 04 & Q-SAT', 'StatusActive', '29.75 ', 'Success')
('SpaceX', 'Pad A, Boca Chica, Texas, USA', 'Tue Aug 04, 2020 23:57 UTC', 'Starship Prototype | 150 Meter Hop', 'StatusActive', '0', 'Success')
('Roscosmos', 'Site 200/39, Baikonur Cosmodrome, Kazakhstan', 'Thu Jul 30, 2020 21:25 UTC', 'Proton-M/Briz-M | Ekspress-80 & Ekspress-103', 'StatusActive', '65.0 ', 'Success')
('ULA', 'SLC-41, Cape Canaveral AFS, Florida, USA', 'Thu Jul 30, 2020 11:50 UTC', 'Atlas V 541 | Perseverance', 'StatusActive', '145.0 ', 'Success')
('CASC', 'LC-9, Taiyuan Satellite Launch Center, China', 'Sat Jul 25, 2020 03:13 UTC', 'Long March 4B | Ziyuan-3 03, Apocalypse-10 & NJU-HKU 1', 'StatusActive', '64.68 ', 'Success')
('

In [ ]:
#*naaaah I'm not :P 
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [ ]:
df.dtypes

Company Name      object
Location          object
Datum             object
Detail            object
Status Rocket     object
Rocket            object
Status Mission    object
dtype: object

In [ ]:
convert_dict = {'Company Name': str,
                'Location': str,
                 'Datum' : str,
                'Detail': str,
                'Status Rocket': str,
                'Rocket': float,
                'Status Mission': str
               }
  
df = df.astype(convert_dict)
df.dtypes

ValueError: ignored

In [ ]:
#Lets find those :P
df[df['Rocket'] == '5,000.0 ']

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
1916,RVSN USSR,"Site 110/37, Baikonur Cosmodrome, Kazakhstan","Tue Nov 15, 1988 03:00 UTC",Energiya/Buran | Buran,StatusRetired,"5,000.0",Success
2000,RVSN USSR,"Site 250, Baikonur Cosmodrome, Kazakhstan","Fri May 15, 1987 17:30 UTC",Energiya/Polyus | Polyus Space Station,StatusRetired,"5,000.0",Success


In [ ]:
df["Rocket"].replace({"5,000.0 ": 5000}, inplace = True)

In [ ]:
df[df['Rocket'] == '5,000.0 ']

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission


In [ ]:
df = df.astype(convert_dict)
df.dtypes

ValueError: ignored

In [ ]:
df["Rocket"].replace({'1,160.0 ': 1160}, inplace = True)
df = df.astype(convert_dict)
df.dtypes

Company Name       object
Location           object
Datum              object
Detail             object
Status Rocket      object
Rocket            float64
Status Mission     object
dtype: object